In [5]:
# 2.6 Data sampling
import tiktoken

# Creating tokenizer
tokenizer = tiktoken.get_encoding("gpt2")

with open ("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

# encoding
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

# removing fist 50 tokens
enc_sample = enc_text[50:]

# Creating input-targets pairs
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x {x}")
print(f"y {y}")

for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "-->", tokenizer.decode([desired]))


5145
x [290, 4920, 2241, 287]
y [4920, 2241, 287, 257]
 and -->  established
 and established -->  himself
 and established himself -->  in
 and established himself in -->  a


In [4]:
# Using DataLoader class from pytoch

import torch
import torch.utils.data as data
import tiktoken


# Creating dataset
class GPTDatasetV1(data.Dataset):
    def __init__(self, txt, tokenizer, max_len, stride = 1):
        super().__init__()
        self.input_ids = []
        self.target_ids = []

        # Getting indexes
        torken_ids = tokenizer.encode(txt)

        # Making shift as in previos example
        for i in range(0, len(torken_ids) - max_len, stride ):
            input_chunk = torken_ids[i: i+max_len]
            target_chunk = torken_ids[i+1 : i + max_len +1 ]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __getitem__(self, index):
        return self.input_ids[index] , self.target_ids[index]
    
    def __len__(self):
        return len(self.input_ids)

# creating Dataloader
tokenizer = tiktoken.get_encoding("gpt2")

with open ("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataset = GPTDatasetV1(raw_text, tokenizer, max_len=4, stride=4)
d_train = data.DataLoader(dataset,batch_size=8, shuffle=False)

data_iter = iter(d_train)
inputs, targets = next(data_iter)
print(f"Inputs \n {inputs} \nTargets \n {targets}")



Inputs 
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]]) 
Targets 
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
